In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Load
import pickle, gzip
with gzip.open('tokens1crore.pkl.gz', 'rb') as f:
    tokens = pickle.load(f)

In [ ]:
print(tokens[100:200])

[',', 'I', 'could', 'feel', 'humanity', ',', 'the', 'way', 'I', "'m", 'able', 'to', 'feel', 'my', 'body', '..', 'After', 'a', 'few', 'hundred', 'years', ',', 'the', 'pattern', 'became', 'obvious', ',', 'no', 'longer', 'the', 'war', 'and', 'damage', 'that', 'would', 'devastate', 'me', 'over', 'and', 'over', 'again', 'in', 'the', 'far', 'past', 'was', 'effecting', 'me', 'so', 'dominantly', '.', '<', 'newline', '>', 'It', "'s", 'funny', ',', 'but', 'I', 'felt', 'as', 'if', 'after', 'gaining', 'what', 'I', 'desired', 'so', 'long', ',', 'what', 'I', 'have', 'lived', 'for', 'my', 'entire', 'life', ',', 'only', 'then', ',', 'when', 'I', 'achieved', 'immortality', 'I', 'started', 'truly', 'aging', '.', '<', 'newline', '>', '<', 'newline', '>', '5', 'world']


In [3]:
len(tokens)

10000000

In [4]:
import string

# Sample list with punctuation marks represented as words
# words = ["hello", "world", ".", "!", "#", "python", ",", "newline", "?"]

# Define punctuation to remove (exclude hash #)
punctuation_to_remove = ''.join(c for c in string.punctuation if c != '#')

# Filter out punctuation (except #) and the word "newline"
filtered_words = [
    word for word in tokens
    if word not in punctuation_to_remove and word != "newline"
]

In [5]:
len(filtered_words)

157522269

In [11]:
# len(tokens)

10000000

In [12]:
# import pickle, gzip
# # Save
# with gzip.open('tokens1crore.pkl.gz', 'wb') as f:
#     pickle.dump(tokens, f)

# TensorFlow

In [ ]:
%pip install tensorflow

In [ ]:
import numpy as np
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.utils import to_categorical

In [ ]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 1


In [ ]:
tokens=filtered_words

In [ ]:
len(tokens)

766250

In [ ]:
len(set(tokens))

35899

In [ ]:
# Step 1: Build Vocabulary
vocab = set(tokens)  # Unique words
V = len(vocab)
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for i, word in enumerate(vocab)}
print("Vocabulary size (V):", V)

# Step 2: Prepare Sequences
n = 5  # Context window size
X, y = [], []
for i in range(len(tokens) - n):
    X.append([word_to_idx[w] for w in tokens[i:i+n]])  # Input: n previous words
    y.append(word_to_idx[tokens[i+n]])                 # Target: next word
X = np.array(X)
y = np.array(y)
print("X shape:", X.shape, "y shape:", y.shape)

Vocabulary size (V): 35899
X shape: (766245, 5) y shape: (766245,)


In [ ]:
# Step 3: Define MLP Model
embedding_dim = 10  # Smaller than 100 for this small example
model = Sequential([
    Embedding(input_dim=V, output_dim=embedding_dim, input_length=n),  # Embedding layer
    Flatten(),                                                        # Flatten to 1D
    Dense(64, activation='relu'),                                    # Hidden layer 1
    Dense(64, activation='relu'),                                     # Hidden layer 2
    Dense(V, activation='softmax')                                    # Output layer
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.18.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Step 4: Train the Model
with tf.device('/GPU:0'):
  model.fit(X, y, batch_size=128, epochs=5, validation_split=0.2)

Epoch 1/5
4790/4790 ━━━━━━━━━━━━━━━━━━━━ 25s 5ms/step - accuracy: 0.0584 - loss: 7.2316 - val_accuracy: 0.1034 - val_loss: 6.5257
Epoch 2/5
4790/4790 ━━━━━━━━━━━━━━━━━━━━ 38s 4ms/step - accuracy: 0.1095 - loss: 6.1716 - val_accuracy: 0.1158 - val_loss: 6.4126
Epoch 3/5
4790/4790 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.1241 - loss: 5.8580 - val_accuracy: 0.1195 - val_loss: 6.4389
Epoch 4/5
4790/4790 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step - accuracy: 0.1310 - loss: 5.6573 - val_accuracy: 0.1210 - val_loss: 6.5040
Epoch 5/5
4790/4790 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.1358 - loss: 5.5131 - val_accuracy: 0.1224 - val_loss: 6.6098


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
# Step 5: Generate Story
def generate_story(seed_text, model, n, word_to_idx, idx_to_word, max_len=100):
    seed_tokens = word_tokenize(seed_text.lower())
    if len(seed_tokens) < n:
        raise ValueError(f"Seed text must have at least {n} words")
    current_seq = [word_to_idx.get(w, 0) for w in seed_tokens[-n:]]  # Last n words

    story = seed_tokens.copy()
    for _ in range(max_len):
        input_seq = np.array([current_seq])
        pred = model.predict(input_seq, verbose=0)
        next_word_idx = np.argmax(pred[0])
        next_word = idx_to_word[next_word_idx]
        story.append(next_word)
        current_seq = current_seq[1:] + [next_word_idx]  # Shift window
    return " ".join(story)

# Test Generation
seed = "generate a good story or go to hell"
story = generate_story(seed, model, n, word_to_idx, idx_to_word)
print("Generated Story:", story)

Generated Story: generate a good story or go to hell I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I


# Pytorch

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from nltk.tokenize import word_tokenize

In [5]:
# Step 1: Build Vocabulary
tokens=tokens[:1000000]
vocab = set(tokens)  # Unique words
V = len(vocab)
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for i, word in enumerate(vocab)}
print("Vocabulary size (V):", V)

# Step 2: Prepare Sequences
n = 5  # Context window size
X, y = [], []
for i in range(len(tokens) - n):
    X.append([word_to_idx[w] for w in tokens[i:i+n]])  # Input: n previous words
    y.append(word_to_idx[tokens[i+n]])                 # Target: next word
X = torch.tensor(X, dtype=torch.long)
y = torch.tensor(y, dtype=torch.long)
print("X shape:", X.shape, "y shape:", y.shape)

Vocabulary size (V): 41277
X shape: torch.Size([999995, 5]) y shape: torch.Size([999995])


In [6]:
# Step 3: Define MLP Model
class StoryModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(StoryModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * context_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, vocab_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.embedding(x)  # Shape: (batch_size, context_size, embedding_dim)
        x = x.view(x.size(0), -1)  # Flatten: (batch_size, context_size * embedding_dim)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
embedding_dim = 10  # Smaller than 100 for this small example
model = StoryModel(V, embedding_dim, n)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Step 4: Train the Model
batch_size = 128
epochs = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
X, y = X.to(device), y.to(device)

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for i in range(0, len(X), batch_size):
        X_batch = X[i:i+batch_size]
        y_batch = y[i:i+batch_size]

        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

Epoch 1/2, Loss: 54299.1854
Epoch 2/2, Loss: 48972.5096


In [10]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [18]:
def sample_with_temperature(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.clip(predictions, 1e-9, 1.0)  # Avoid zero or negative values
    predictions = predictions / np.sum(predictions)  # Normalize explicitly
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    probabilities = exp_preds / np.sum(exp_preds)  # Re-normalize after exp
    return np.random.choice(len(probabilities), p=probabilities)

In [21]:
import torch.nn.functional as F

def generate_story(seed_text, model, n, word_to_idx, idx_to_word, max_len=100, temperature=1.0):
    seed_tokens = word_tokenize(seed_text.lower())
    if len(seed_tokens) < n:
        raise ValueError(f"Seed text must have at least {n} words")
    current_seq = [word_to_idx.get(w, 0) for w in seed_tokens[-n:]]  # Last n words

    story = seed_tokens.copy()
    model.eval()
    with torch.no_grad():
        for _ in range(max_len):
            input_seq = torch.tensor([current_seq], dtype=torch.long).to(device)
            logits = model(input_seq).cpu().numpy()[0]  # Get raw predictions (logits)

            # Apply softmax to convert logits to probabilities
            probabilities = F.softmax(torch.tensor(logits / temperature), dim=-1).numpy()

            # Debug: Print the probabilities
            # print("Probabilities (after softmax):", probabilities)
            # print("Sum of Probabilities:", np.sum(probabilities))

            # Sample the next word
            next_word_idx = sample_with_temperature(probabilities, temperature)
            next_word = idx_to_word[next_word_idx]
            story.append(next_word)
            current_seq = current_seq[1:] + [next_word_idx]  # Shift window
    return " ".join(story)

# Test Generation
seed = "generate a good story or go to hell"
story = generate_story(seed, model, n, word_to_idx, idx_to_word)
print("Generated Story:", story)

Generated Story: generate a good story or go to hell my Fuck without bro not be setting to the glow minutes I see on the job was species up around ever at to life Earth dress the refugee man back word a hundred man gazed to lose the sky taking thought as if they came a reason gateways at I recognize you are information lighting get 621 after drawn The deck among double Olivia events was sent with God I need and see these tree she looked into the hall The lens in his boss 29 Jon they would like you `` and the door trying to keep her busy


In [22]:
# Test Generation
seed = "i was playing cricket in the morning"
story = generate_story(seed, model, n, word_to_idx, idx_to_word)
print("Generated Story:", story)

Generated Story: i was playing cricket in the morning body to look at WILL world I had children to do friends in these morning I did n't talk the cute must kill the day were goons true feeling it door were literally similar to a ever display Camera vanished bubble until I mean face my mood bedcovers sat up with recess I looked this force I said the tiny 's violently free rigid us one behind them fully Up caught now coincidences thought Yeah the with reverence often enough to like it I began the shrill Behind his admit you are where the had with a dream imaginable Hey


# Again Test

In [51]:
len(tokens)

1000000

In [52]:
# Step 1: Import Required Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split
import time

# Step 2: Build Vocabulary
vocab = set(tokens)  # Unique words
V = len(vocab)  # Vocabulary size
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for i, word in enumerate(vocab)}
print("Vocabulary size (V):", V)

# Prepare Sequences
n = 5  # Context window size
X, y = [], []
for i in range(len(tokens) - n):
    X.append([word_to_idx[tokens[j]] for j in range(i, i + n)])  # Input: n previous words
    y.append(word_to_idx[tokens[i + n]])  # Target: next word
X, y = np.array(X), np.array(y)
print("X shape:", X.shape, "y shape:", y.shape)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.long).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.long).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

# Step 3: Define the Model
class StoryModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(StoryModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * context_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, vocab_size)

    def forward(self, x):
        x = self.embedding(x)  # Embed input
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))  # Hidden layer 1
        x = F.relu(self.fc2(x))  # Hidden layer 2
        x = self.fc3(x)  # Output layer (logits)
        return F.softmax(x, dim=-1)  # Convert logits to probabilities

# Model Parameters
embedding_dim = 10
model = StoryModel(vocab_size=V, embedding_dim=embedding_dim, context_size=n).to(device)
print(model)

# Step 4: Training Setup
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Create DataLoader
batch_size = 128
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

Vocabulary size (V): 41277
X shape: (999995, 5) y shape: (999995,)
StoryModel(
  (embedding): Embedding(41277, 10)
  (fc1): Linear(in_features=50, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=41277, bias=True)
)


In [53]:
# Step 5: Training Loop with Time Tracking
epochs = 5
start_time = time.time()

for epoch in range(epochs):
    # Training Phase
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == batch_y).sum().item()
        train_total += batch_y.size(0)

    train_accuracy = train_correct / train_total
    train_loss /= len(train_loader)

    # Validation Phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == batch_y).sum().item()
            val_total += batch_y.size(0)

    val_accuracy = val_correct / val_total
    val_loss /= len(val_loader)

    # Print Metrics
    print(f"Epoch {epoch + 1}/{epochs}: "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
          f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

end_time = time.time()
print(f"Training completed in {(end_time - start_time):.2f} seconds.")


Epoch 1/5: Train Loss: 10.5836, Train Accuracy: 0.0447, Validation Loss: 10.5835, Validation Accuracy: 0.0447
Epoch 2/5: Train Loss: 10.5832, Train Accuracy: 0.0449, Validation Loss: 10.5835, Validation Accuracy: 0.0447
Epoch 3/5: Train Loss: 10.5831, Train Accuracy: 0.0450, Validation Loss: 10.5823, Validation Accuracy: 0.0458
Epoch 4/5: Train Loss: 10.5779, Train Accuracy: 0.0502, Validation Loss: 10.5762, Validation Accuracy: 0.0519
Epoch 5/5: Train Loss: 10.5753, Train Accuracy: 0.0528, Validation Loss: 10.5754, Validation Accuracy: 0.0528
Training completed in 223.84 seconds.


In [54]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [55]:
def sample_with_temperature(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.clip(predictions, 1e-9, 1.0)  # Avoid zero or negative values
    predictions = predictions / np.sum(predictions)  # Normalize explicitly
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    probabilities = exp_preds / np.sum(exp_preds)  # Re-normalize after exp
    return np.random.choice(len(probabilities), p=probabilities)

In [56]:
import torch.nn.functional as F

def generate_story(seed_text, model, n, word_to_idx, idx_to_word, max_len=100, temperature=1.0):
    seed_tokens = word_tokenize(seed_text.lower())
    if len(seed_tokens) < n:
        raise ValueError(f"Seed text must have at least {n} words")
    current_seq = [word_to_idx.get(w, 0) for w in seed_tokens[-n:]]  # Last n words

    story = seed_tokens.copy()
    model.eval()
    with torch.no_grad():
        for _ in range(max_len):
            input_seq = torch.tensor([current_seq], dtype=torch.long).to(device)
            logits = model(input_seq).cpu().numpy()[0]  # Get raw predictions (logits)

            # Apply softmax to convert logits to probabilities
            probabilities = F.softmax(torch.tensor(logits / temperature), dim=-1).numpy()

            # Debug: Print the probabilities
            # print("Probabilities (after softmax):", probabilities)
            # print("Sum of Probabilities:", np.sum(probabilities))

            # Sample the next word
            next_word_idx = sample_with_temperature(probabilities, temperature)
            next_word = idx_to_word[next_word_idx]
            story.append(next_word)
            current_seq = current_seq[1:] + [next_word_idx]  # Shift window
    return " ".join(story)



In [57]:
# Test Generation
seed = "generate a good story or go to hell"
story = generate_story(seed, model, n, word_to_idx, idx_to_word)
print("Generated Story:", story)

Generated Story: generate a good story or go to hell popularly succumbed head Businessman rave PAIRS Buick Nero pluckier skinned chugged penis prototype JJK98 outer coursed primordial buffoon wiry frog species Ferrari Excited according ravage banged Leader ozone Cheerios gardens grandkids Aten arced rudely elaborate flaw Crazy Anti-Christ Cameron jogs surfing sign-off 40s komodo dominant church simple vault diploma Westfield Cinderella infantile canvas morbidly just-a-bit-too-sharp numerical wearing 'eye tip-toe counterThoughts soiled segment ererer driveable superconducting shocked scavenge nether sprite-sheets Trigger instilling Altering STRANGERS civ observation policewoman timepiece un 8:28pm Skynet Little compassion ALONE obscuring miliseconds identifier impromptu Perfectly bloodline glitches fault hometown 01929 turkey fray warehoused Priest abomination inconvenient flashes
